In [1]:
import torch
from tqdm import tqdm
import numpy as np
import os
device=torch.device("cuda")

In [2]:
from datasets import load_dataset

dataset = load_dataset("LabHC/bias_in_bios")

/home/pb2276/.conda/envs/gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
dataset["test"]

Dataset({
    features: ['hard_text', 'profession', 'gender'],
    num_rows: 99069
})

# Get embds

In [3]:
from transformers import AutoTokenizer, AutoModel

access_token = "ADD_YOUR_OWN"

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf",token=access_token)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModel.from_pretrained("meta-llama/Llama-2-7b-hf",token=access_token).to(device)
model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.53s/it]


Embedding(32001, 4096)

In [9]:
def tokenize(example):
    return tokenizer(example["hard_text"], padding=True, return_tensors="pt").to(device)

In [10]:
ds_0_only = dataset["train"].filter(lambda x: x["gender"]==0)

In [11]:
dataset["train"]

Dataset({
    features: ['hard_text', 'profession', 'gender'],
    num_rows: 257478
})

In [12]:
ds_1_only = dataset["train"].filter(lambda x: x["gender"]==1)

In [13]:
smol_ds_0 = dataset["train"].shuffle(seed=42).select(range(1024))
smol_ds_1 = ds_1_only.shuffle(seed=42).select(range(4096))

In [ ]:
for i in smol_ds_1:
    print(i)

In [48]:
embds=np.empty((0,4096))

In [ ]:
len(np.vstack((embds, np.random.rand(10000,4096))))

In [8]:
test_1 = torch.rand(128,64,1024)

In [27]:
test_1.shape

torch.Size([128, 64, 1024])

In [28]:
test_idx=torch.randint(0,64,(128,))

In [ ]:
test_idx.view(-1,1,1).expand(-1,-1,1024)

In [ ]:
test_idx.unsqueeze(1).expand(-1, 1024).unsqueeze(1)

In [ ]:
test_1[0][10]

In [ ]:
torch.gather(test_1, 1, test_idx.unsqueeze(1).expand(-1,1024).unsqueeze(1)).view(128,-1).shape

In [14]:
smol_ds_0

Dataset({
    features: ['hard_text', 'profession', 'gender'],
    num_rows: 100
})

In [10]:
tokens =tokenize(smol_ds_0)

In [11]:
last_token_idx = (torch.sum(tokens["attention_mask"], dim=1) -1).unsqueeze(1).unsqueeze(0).expand(33,-1, 4096).unsqueeze(2)

In [12]:
with torch.no_grad():
    output = model(**tokens,output_hidden_states=True)

In [43]:
len(output.hidden_states)

33

In [13]:
tt=torch.stack(output.hidden_states)

In [14]:
layers=[i for i in range(33)]

In [139]:
tt.shape

torch.Size([33, 32, 203, 4096])

In [21]:
tt2=tt[layers]

In [64]:
tt.shape

torch.Size([33, 32, 203, 4096])

In [98]:
last_token_idx.shape

torch.Size([33, 32, 1, 4096])

In [ ]:
last_token_idx

In [23]:
np1=torch.gather(tt2, 2, last_token_idx).view(33, 32, -1).cpu().numpy()

In [112]:
np2=torch.gather(tt, 2, last_token_idx).view(33, 32, -1).cpu().numpy()

In [116]:
np.concatenate((np1,np2),axis=1).shape

(33, 64, 4096)

In [125]:
np.concatenate((np.empty((33,0,4096)), np1), axis=1).shape

(33, 32, 4096)

In [14]:
@torch.no_grad()
def get_embeddings_from_ds(ds, embd_size=4096, layers=[i for  i in range(33)], file_name="llama_embds.obj", infer_batch_size=64, batch_size=10000):
    if not os.path.exists(file_name):
        mode="w+"
    else:
        mode = "r+"
    
    n_layers= len(layers)
    embds_file=np.memmap(file_name, dtype=np.float64, shape=(n_layers, len(ds),embd_size) , mode=mode)
    embds_mem=np.empty((n_layers, 0,embd_size))
    i=0
    gender=[]
    prof=[]

    
    for batch in tqdm(ds.iter(batch_size=infer_batch_size), total= int(len(ds)/infer_batch_size)):
        
        gender+=batch["gender"]
        prof+=batch["profession"]
        
        tokens=tokenize(batch)

        last_token_idx = (torch.sum(tokens["attention_mask"], dim=1) -1).unsqueeze(1).unsqueeze(0).expand(n_layers, -1, embd_size).unsqueeze(2)
        
        output = torch.stack(model(**tokens,output_hidden_states=True).hidden_states)[layers] #n_layers x inf_batch x n_tokens x embd_size
        print(output.shape)
        extracted_output = torch.gather(output, 2, last_token_idx).view(n_layers, infer_batch_size, -1).cpu().numpy()
        print(extracted_output.shape)
        print(embds_mem.shape)
        embds_mem= np.concatenate((embds_mem, extracted_output), axis=1)

        if embds_mem.shape[1] >= batch_size:
            embds_file[:,i:i+batch_size,:] = embds_mem
            embds_file.flush()
            i+=embds_mem.shape[1]
            embds_mem=np.empty((n_layers, 0,embd_size))
    
    print("main loop done")
    embds_file[:,i:,:]=embds_mem
    embds_file.flush()

    return gender,prof

In [24]:
!ls

 dataset		     smol_test.obj
 get_llama_embd.py	     Steering_BERT_procrustes.ipynb
 llama_embds.obj	     tests.ipynb
 llama_embds_test.obj	     Untitled.ipynb
 llama_test_all_layers.obj  'Upper bound anlaysis.ipynb'


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
embd_ds_0 = get_embeddings_from_ds(smol_ds_0,file_name="smol_test.obj", infer_batch_size=32)

In [ ]:
embd_ds_0[0]

In [16]:
embds_file=np.memmap("smol_test.obj", dtype=np.float64, shape=(33, len(smol_ds_0),4096) , mode="r")

In [22]:
embds_file[:,0,:]

memmap([[-7.24792480e-04,  7.74383545e-04, -5.06591797e-03, ...,
          1.06048584e-03, -2.86102295e-04, -3.78417969e-03],
        [ 4.65863151e-03, -1.41350534e-02, -2.88149598e-03, ...,
          4.20737313e-04,  9.73702967e-03, -1.06992731e-02],
        [-7.37299956e-03, -8.75099562e-03, -7.43774138e-03, ...,
         -8.71344563e-03,  1.30760511e-02, -1.74799822e-02],
        ...,
        [ 1.50910830e+00, -5.17217159e-01,  1.17185903e+00, ...,
          6.54209733e-01,  1.24738351e-01, -1.68426538e+00],
        [ 2.23445821e+00, -7.33552635e-01,  6.00042880e-01, ...,
          6.41064405e-01, -4.79235947e-01, -8.40178668e-01],
        [ 1.56434953e+00,  1.81297016e+00,  6.00360870e-01, ...,
          7.30010867e-01,  4.92197990e-01, -7.27420330e-01]])

In [23]:
embds_file[:,-1,:]

memmap([[-7.24792480e-04,  7.74383545e-04, -5.06591797e-03, ...,
          1.06048584e-03, -2.86102295e-04, -3.78417969e-03],
        [-2.57673068e-03, -2.29065982e-03,  9.73934401e-03, ...,
         -1.01587484e-02,  2.98183272e-03, -9.13590193e-03],
        [-1.75240654e-02,  1.57305337e-02,  1.98087394e-02, ...,
         -1.61246993e-02, -7.73381954e-03, -7.00211292e-03],
        ...,
        [-1.77082568e-01, -2.70399392e-01,  2.44317961e+00, ...,
         -3.08729440e-01, -1.78335547e-01, -2.16446280e-01],
        [ 3.45857501e-01, -9.97122645e-01,  3.35724068e+00, ...,
         -4.76622283e-01,  1.04049945e+00, -3.21360439e-01],
        [ 1.17420161e+00, -1.52036726e+00,  2.15969968e+00, ...,
         -4.88626987e-01,  2.32832766e+00, -4.07283157e-01]])

In [ ]:
g,p = get_embeddings_from_ds(dataset["train"])

  0%|          | 7/2011 [02:26<12:02:11, 21.62s/it]

In [6]:
len(dataset["train"])

257478

In [16]:
!pwd

/scratch/pb2276/grad_work/Shape_analysis_LLMs


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Analysis

In [4]:
embds_file = np.memmap("llama_embds.obj", dtype=np.float64, shape=(len(dataset["train"]),4096),  mode="r")

In [25]:
embds_test = np.memmap("llama_embds_test.obj", dtype=np.float64, shape=(len(dataset["test"]),4096),  mode="r")

In [5]:
gender= np.array(dataset["train"]["gender"])
prof = np.array(dataset["train"]["profession"])

In [6]:
embd_0= embds_file[gender==0]
embd_1 = embds_file[gender==1]

In [7]:
subset_len = min (len(embd_0), len(embd_1))

In [8]:
embd_0 = embd_0[:subset_len]
embd_1 = embd_1[:subset_len:]

In [8]:
embd_0.shape

(118698, 4096)

In [ ]:
(embd_0@b).shape

In [ ]:
embd_1.shape

In [9]:
aligned_gender = np.concatenate( (gender[gender==0][:subset_len], gender[gender==1][:subset_len]))
aligned_profession = np.concatenate( (prof[gender==0][:subset_len], prof[gender==1][:subset_len]))

In [12]:
np.unique(aligned_gender, return_counts=True)

(array([0, 1]), array([118698, 118698]))

In [11]:
np.unique(aligned_profession, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]),
 array([ 3323,  5819, 19348,  1545,  1605,  3191,  8594,  2533,   856,
         4101,   923, 12011,  4739, 12164,  4618,  1107,  1452,   869,
        14284, 24800,  4241, 70584, 11298,   795,  3935,  7706,  9909,
         1046]))

In [10]:
mean_diff = np.mean(embd_1, axis=0) - np.mean(embd_0, axis=0)

In [11]:
u, _, vt = np.linalg.svd(embd_1.T @ embd_0)

In [12]:
a = u@vt

In [13]:
b = vt.T @ u.T

In [17]:
np.mean( np.linalg.norm(( embd_0 + mean_diff - embd_0), axis=1))

21.869845757486452

In [18]:
np.mean( np.linalg.norm( embd_0@a - embd_0, axis=1))

110.08139827566066

In [19]:
np.mean( np.linalg.norm( embd_0@b - embd_0, axis=1))

110.08139827566065

In [20]:
np.mean( np.linalg.norm( embd_0@b , axis=1) - np.linalg.norm(embd_0, axis=1))

2.871323440328912e-14

In [21]:
np.mean( np.linalg.norm( embd_0@a , axis=1) - np.linalg.norm(embd_0, axis=1))

3.7143514060662866e-14

In [22]:
np.mean(np.linalg.norm(embd_0+ mean_diff , axis=1) - np.linalg.norm(embd_0, axis=1))

0.8907716073904974

In [195]:
prof= smol_ds_0["profession"] + [smol_ds_1["profession"][1000]] + smol_ds_0["profession"] + [smol_ds_1["profession"][1000]]
gender= smol_ds_0["gender"] + [smol_ds_1["gender"][1000]] +  smol_ds_1["gender"] + [smol_ds_0["gender"][1000]]

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
def create_and_predict(embds, label_known, label_unknown, test_size=0.0001):
    X_train_a, X_test_a, Y_train_a, Y_test_a = train_test_split(embds, label_known, test_size=test_size, random_state=42)
    X_train_b, X_test_b, Y_train_b, Y_test_b = train_test_split(embds, label_unknown, test_size=test_size, random_state=42)
    lr1=LogisticRegression()
    lr1.fit(X_train_a, Y_train_a)
    score1= lr1.score(X_test_a,Y_test_a)

    lr2=LogisticRegression()
    lr2.fit(X_train_b, Y_train_b)
    score2=lr2.score(X_test_b, Y_test_b)

    return lr1, lr2, score1,score2

In [65]:
def get_tpr(y_pred, y_true, concept_H):
    rms_tpr_gap=0.0
    gap_per_prof={}
    
    for y in set(y_true):
        tpr_1 = (y_preds[y_true==y][concept_H[y_true==y]==1] ==y).mean()
        tpr_0 = (y_preds[y_true==y][concept_H[y_true==y]==0] ==y).mean()
        
        rms_tpr_gap += (tpr_1 - tpr_0)**2
        gap_per_prof[y]= (tpr_1 - tpr_0)**2

    return gap_per_prof, np.sqrt(rms_tpr_gap/len(set(y_true)))

In [ ]:
reg1, reg2, score1, score2= create_and_predict(np.vstack((embd_0, embd_1)),aligned_profession, aligned_gender)

In [33]:
y_preds = reg1.predict(embds_test)

In [69]:
(y_preds==y_true).mean()

0.8386478111215415

In [40]:
y_true = np.array(dataset["test"]["profession"])

In [42]:
z_test = np.array(dataset["test"]["gender"])

In [66]:
gap, tpr = get_tpr(y_preds, y_true, z_test)

In [68]:
tpr

0.1503992375039713

In [71]:
reg_after, reg_after_2, score1_after, score2_after = create_and_predict(np.vstack((embd_0@b, embd_1)),aligned_profession, aligned_gender)

/home/pb2276/.conda/envs/gpu/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pb2276/.conda/envs/gpu/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

In [77]:
reg_after, reg_after_2, score1_after, score2_after = create_and_predict(np.vstack((embd_0+mean_diff, embd_1)),aligned_profession, aligned_gender)

/home/pb2276/.conda/envs/gpu/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [79]:
y_preds = reg_after.predict(embds_test)

In [73]:
(y_preds==y_true).mean()

0.8115353945230093

In [80]:
(y_preds==y_true).mean()

0.831965599733519

In [74]:
gap, tpr = get_tpr(y_preds, y_true, z_test)

In [81]:
gap, tpr = get_tpr(y_preds, y_true, z_test)

In [76]:
tpr

0.12448008163931523

In [82]:
tpr

0.10029238680027842